In [37]:
import urllib
import json
from IPython.display import JSON

from datetime import datetime

import glob
import os

Input definition

In [2]:
burstId = 249435
sub_swath = 'IW2'
start_date = '2024-08-01'
end_date = '2024-09-30'

with open("CDSE_SECRET", "r") as file:
    lines = file.readlines()
CDSE_ACCESS_KEY = lines[0].strip().split(": ")[1]
CDSE_SECRET_KEY = lines[1].strip().split(": ")[1]

output_folder = '/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/'

## Burst download

HTTP queries to extract all the data of interest

In [3]:
https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
    f"ContentDate/Start ge {start_date}T00:00:00.000Z and ContentDate/Start le {end_date}T23:59:59.000Z and "
    f"PolarisationChannels eq 'VV' and "
    f"BurstId eq {burstId} and "
    f"SwathIdentifier eq '{sub_swath}'"
) + "&$top=1000"

with urllib.request.urlopen(https_request) as response:
    content = response.read().decode()
bursts = json.loads(content)
JSON(bursts)

<IPython.core.display.JSON object>

For each data run the cdse utilities to extract the burst of interest

In [4]:
for b in bursts['value']:
    
    burst_extract_cmd = (
        f"docker run -v {output_folder}:/home/ubuntu "
        f"-e AWS_ACCESS_KEY_ID={CDSE_ACCESS_KEY} -e AWS_SECRET_ACCESS_KEY={CDSE_SECRET_KEY} "
        f"cdse_utilities sentinel1_burst_extractor.sh -o /home/ubuntu "
        f"-n {b['ParentProductName']} -p vv -s {sub_swath.lower()} -r {burstId}"
    )

    #print(burst_extract_cmd)
    os.system(burst_extract_cmd)


## Sentinel-1 processing

### 1) SNAP-InSAR

Input definition

In [21]:
graph_filename = './graphs/coh_2images_GeoTiff.xml'

InSAR_pairs = [
    '20240809_20240821',
    '20240809_20240902',
    '20240821_20240902',
    '20240821_20240914',
    '20240902_20240914'
]

In [ ]:
for pair in InSAR_pairs:

    mst_filename = glob.glob(os.path.join(output_folder, f'*{pair.split("_")[0]}*', 'manifest.safe'))[0]
    slv_filename = glob.glob(os.path.join(output_folder, f'*{pair.split("_")[1]}*', 'manifest.safe'))[0]
    output_filename = os.path.join(output_folder, 'S1_coh', f'S1_coh_{pair}.tif')

    cmd = f'gpt "{graph_filename}" -Pmst_filename="{mst_filename}" -Pslv_filename="{slv_filename}" -Poutput_filename="{output_filename}"'

    #print(cmd)
    os.system(cmd)

### 2) SNAP-preprocessing

Input definition

In [60]:
graph_mst_filename = './graphs/pre-processing_2images_SaveMst_GeoTiff.xml'
graph_slv_filename = './graphs/pre-processing_2images_SaveOnlySlv_GeoTiff.xml'
output_folder_preprocessing = os.path.join(output_folder, 'S1_slc')

mst_date = '20240809'
mst_filename = glob.glob(os.path.join(output_folder, f'*{mst_date}*', 'manifest.safe'))[0]

slv_filename_list = glob.glob(os.path.join(output_folder, f'*SAFE', 'manifest.safe'))
slv_filename_list.remove(mst_filename)

Run the first preprocessing saving the master with the associated lat/lon grid

In [62]:
mst_bandname = f'{sub_swath}_VV_mst_{datetime.strptime(mst_date, "%Y%m%d").strftime("%d%b%Y")}'

slv_filename = slv_filename_list[0]
slv_date = os.path.basename(os.path.dirname(slv_filename)).split('_')[2][:8]
slv_bandname = f'{sub_swath}_VV_slv1_{datetime.strptime(slv_date, "%Y%m%d").strftime("%d%b%Y")}'

output_mst_filename = os.path.join(output_folder_preprocessing, f'S1_slc_{mst_date}.tif')
output_slv_filename = os.path.join(output_folder_preprocessing, f'S1_slc_{slv_date}.tif')

cmd = (
    f'gpt "{graph_mst_filename}" -Pmst_filename="{mst_filename}" -Pslv_filename="{slv_filename}" '
    f'-Pi_q_mst_bandnames="i_{mst_bandname},q_{mst_bandname}" '
    f'-Pi_q_slv_bandnames="i_{slv_bandname},q_{slv_bandname}" '
    f'-Poutput_mst_filename="{output_mst_filename}" -Poutput_slv_filename="{output_slv_filename}"'
)

print(cmd)
os.system(cmd)

gpt "./graphs/pre-processing_2images_SaveMst_GeoTiff.xml" -Pmst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1A_SLC_20240809T170739_249435_IW2_VV_440377.SAFE/manifest.safe" -Pslv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1A_SLC_20240821T170739_249435_IW2_VV_442005.SAFE/manifest.safe" -Pi_q_mst_bandnames="i_IW2_VV_mst_09Aug2024,q_IW2_VV_mst_09Aug2024" -Pi_q_slv_bandnames="i_IW2_VV_slv1_21Aug2024,q_IW2_VV_slv1_21Aug2024" -Poutput_mst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1_slc/S1_slc_20240809.tif" -Poutput_slv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1_slc/S1_slc_20240821.tif"


0

Run the preprocessing for the remaining slave images

In [65]:
for slv_filename in slv_filename_list[1:]:

    slv_date = os.path.basename(os.path.dirname(slv_filename)).split('_')[2][:8]
    slv_bandname = f'{sub_swath}_VV_slv1_{datetime.strptime(slv_date, "%Y%m%d").strftime("%d%b%Y")}'
    output_slv_filename = os.path.join(output_folder_preprocessing, f'S1_slc_{slv_date}.tif')

    cmd = (
        f'gpt "{graph_slv_filename}" -Pmst_filename="{mst_filename}" -Pslv_filename="{slv_filename}" '
        f'-Pi_q_slv_bandnames="i_{slv_bandname},q_{slv_bandname}" '
        f'-Poutput_slv_filename="{output_slv_filename}"'
    )
    
    print(cmd)
    os.system(cmd)

gpt "./graphs/pre-processing_2images_SaveOnlySlv_GeoTiff.xml" -Pmst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1A_SLC_20240809T170739_249435_IW2_VV_440377.SAFE/manifest.safe" -Pslv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1A_SLC_20240914T170740_249435_IW2_VV_445439.SAFE/manifest.safe" -Pi_q_slv_bandnames="i_IW2_VV_slv1_14Sep2024,q_IW2_VV_slv1_14Sep2024" -Poutput_slv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1_slc/S1_slc_20240914.tif"
gpt "./graphs/pre-processing_2images_SaveOnlySlv_GeoTiff.xml" -Pmst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1A_SLC_20240809T170739_249435_IW2_VV_440377.SAFE/manifest.safe" -Pslv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/S1A_SLC_20240926T170740_249435_IW2_VV_447213.SAFE/manifest.safe" -Pi_q_slv_bandnames="i_IW2_VV_slv1_26Sep2024,q_IW2_VV_slv1_26Sep2024" -Poutput_slv_filename="/home/mcallegari/PROJECT_DAT